<img src="https://github.com/Databricks-BR/lab_genai/blob/main/img/header.png?raw=true" width=100%>

# Hands-On LAB 03 - Usando Batch Inference
## Análise de sentimento, extração de entidades e geração de texto

Treinamento Hands-on na plataforma Databricks com foco nas funcionalidades de IA Generativa.

## Objetivos do Exercício

O objetivo desse laboratório é implementar o seguinte caso de uso:

### Aumentando a satisfação do cliente com análise automática de avaliações

Neste laboratório, construiremos um pipeline de dados que pega avaliações de clientes, na forma de texto livre e as enriquece com informações extraídas ao fazer perguntas em linguagem natural aos modelos de IA Generativa disponíveis no Databricks. Também forneceremos recomendações para as próximas melhores ações à nossa equipe de atendimento ao cliente - ou seja, se um cliente requer acompanhamento e um rascunho de mensagem de resposta.

Para cada avaliação, nós:

Identificamos o sentimento do cliente e extraímos os produtos mencionados
- Geramos uma resposta personalizada para o cliente

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-query-function-review.png" width="100%">

## Preparação

Para executar os exercícios, precisamos conectar este notebook a um compute.

Basta seguir os passos abaixo:
1. No canto superior direito, clique em **Connect**
2. Selecione o **SQL Warehouse** desejado

## Exercício 02.01 - Acessando o conjunto de dados

Agora, vamos acessar as avaliações de produto que carregamos no laboratório anterior.

Neste laboratório iremos utilizar duas tabelas:
- **Avaliações**: dados não-estruturados com o conteúdo das avaliações
- **Clientes**: dados estruturados como o cadastro e consumo dos clientes

Agora, vamos visualizar estes dados!

### A. Selecionar o database que criamos anteriormente

In [0]:
USE academy.playground

### B. Visualizar a tabela de avaliações

In [0]:
SELECT * FROM avaliacoes

### C. Visualizar a tabela de clientes

In [0]:
SELECT * FROM clientes

## Exercício 02.02 - Analisando sentimento e extraindo informações

Nosso objetivo é permitir a análise rápida de grandes volumes de avaliações de forma rápida e eficiente. Para isso, precisamos extrair as seguintes informações:

- Produtos mencionados
- Sentimento do cliente
- Caso seja negativo, qual o motivo da insatisfação

Vamos ver como podemos aplicar IA Generativa para acelerar nosso trabalho.

### 1. Usando Foundation Models

<img src="https://docs.databricks.com/en/_images/serving-endpoints-list.png" style="float: right; padding-left: 10px; padding-top: 15px" width=600>

Primeiro, precisamos de um modelo capaz de interpretar o texto das avaliações e extrair as informações desejadas. Para isso, vamos utilizar **[Foundation Models](https://docs.databricks.com/en/machine-learning/foundation-models/index.html#pay-per-token-foundation-model-apis)**, que são grandes modelos de linguagem (LLMs) servidos pela Databricks e que podem ser consultados sob-demanda sem a necessidade de implantação ou gerenciamento desses recursos.

Alguns modelos disponíveis são:

- Llama 3.1 48B Instruct
- Llama 3.3 70B Instruct
- Claude Sonnet 4
- GPT-5
- Gemini 2.5 Flash
- GPT OSS 120B
- GTE Large

Agora, vamos vê-los em funcionamento!

1. No **menu principal** à esquerda, clique em **`Serving`**
2. No card do modelo **Meta Llama 3.1 8B Instruct**, clique em **`Use`**
3. Adicione a instrução abaixo:
    ```
    Classifique o sentimento da seguinte avaliação:
    Comprei um tablet e estou muito insatisfeito com a qualidade da bateria. Ela dura muito pouco tempo e demora muito para carregar.
     ```
    <br>
 4. Clique no ícone **enviar**

Com isso, já conseguimos de forma rápida começar a prototipar nossos novos produtos de dados!

### 2. Comparando modelos no AI Playground

<img src="https://docs.databricks.com/en/_images/ai-playground.gif" style="float: right; padding-left: 10px" width=600>

Para decidir qual o melhor modelo e instrução para o nosso caso de uso, podemos utilizar o **[AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html)**.

Assim, podemos testar rapidamente diversas combinações de modelos e instruções através de uma interface intuitiva e escolher a melhor opção para utilizarmos no nosso projeto.

Vamos fazer o seguinte teste:

1. No **menu principal** à esquerda, clique em **`Playgroud`**
2. Clique no **seletor de modelos** e selecione o modelo **`Meta Llama 3.1 8B Instruct`** (caso já não esteja selecionado)
3. Clique no ícone **`Add endpoint`**
4. Clique no **seletor de modelos** e selecione o modelo **`Meta Llama 3.3 70B Instruct`**
5. Clique no ícone **`Add endpoint`**
6. Clique no **seletor de modelos** e selecione o modelo **`Llama 4 Maverick`**
7. Adicione a instrução abaixo:
  ```
    Classifique o sentimento da seguinte avaliação:
    Comprei um tablet e estou muito insatisfeito com a qualidade da bateria. Ela dura muito pouco tempo e demora muito para carregar.
  ```
<br>
8. Clique no ícone **enviar**

Agora, podemos comparar as respostas, o tempo e o custo de cada um dos modelos para escolher aquele que melhor atende às necessidades do nosso projeto!

### 3. Usando AI Functions

Por fim, para que possamos escalar a utilização dos modelos de IA Generativa, podemos utilizar **[AI Functions](https://docs.databricks.com/en/large-language-models/ai-functions.html)**.

Estas permitem executar modelos de IA Generativa sobre nossos bancos de dados corporativos diretamente em consultas SQL, uma linguagem amplamente utiliza por analistas de dados e de negócio. Com isso, também podemos criar novas tabelas com as informações extraídas para serem utilizadas em nossas análises.

Existem funções nativas para executar tarefas pré-definidas ou enviar qualquer instrução desejada para ser executada. Seguem as descrições abaixo:

| Gen AI SQL Function | Descrição |
| -- | -- |
| [ai_analyze_sentiment](https://docs.databricks.com/pt/sql/language-manual/functions/ai_analyze_sentiment.html) | Análise de Sentimento |
| [ai_classify](https://docs.databricks.com/pt/sql/language-manual/functions/ai_classify.html) | Classifica o texto de acordo com as categorias definidas |
| [ai_extract](https://docs.databricks.com/pt/sql/language-manual/functions/ai_extract.html) | Extrai as entidades desejadas |
| [ai_fix_grammar](https://docs.databricks.com/pt/sql/language-manual/functions/ai_fix_grammar.html) | Corrige a gramática do texto fornecido |
| [ai_gen](https://docs.databricks.com/pt/sql/language-manual/functions/ai_gen.html) | Gera um novo texto conforme a instrução | 
| [ai_mask](https://docs.databricks.com/pt/sql/language-manual/functions/ai_mask.html) | Mascara dados sensíveis |
| [ai_query](https://docs.databricks.com/pt/sql/language-manual/functions/ai_query.html) | Envia instruções para o modelo desejado |
| [ai_similarity](https://docs.databricks.com/pt/sql/language-manual/functions/ai_similarity.html) | Calcula a similaridade entre duas expressões |
| [ai_summarize](https://docs.databricks.com/pt/sql/language-manual/functions/ai_summarize.html) | Sumariza o texto fornecido |
| [ai_translate](https://docs.databricks.com/pt/sql/language-manual/functions/ai_translate.html) | Traduz o texto fornecido |

Para extrair as informações que precisamos, vamos utilizar algumas dessas funções abaixo!

#### A. Análise de sentimento

In [0]:
SELECT
  avaliacao,
  ai_analyze_sentiment(avaliacao) as sentimento
FROM
  avaliacoes
LIMIT 10

#### B. Extração dos produtos mencionados

In [0]:
SELECT
  avaliacao,
  ai_extract(
    avaliacao,
    array('produto')
  ) AS produtos_mencionados
FROM avaliacoes
LIMIT 10;

#### C. Extração do motivo da insatisfação

*DICA: use a função AI_QUERY() para fornecer um prompt customizado*

In [0]:
SELECT
  avaliacao,
  ai_query(
    "databricks-meta-llama-3-3-70b-instruct",
    avaliacao,
    'Extraia o motivo da insatisfação do cliente.'
  ) AS motivo_insatisfacao
FROM avaliacoes
LIMIT 10;

### 4. Analisando sentimento e extraindo entidades em escala

Ter que especificar as instruções várias vezes acaba sendo trabalhoso, especialmente para Analistas de Dados que deveriam focar em analisar os resultados dessa extração.

Para simplificar o acesso à essa inteligência, criaremos uma função SQL para encapsular esse processo e poder apenas informar em qual coluna do nosso conjunto de dados gostaríamos de aplicá-la.

Aqui, vamos aproveitar para enviar uma única consulta ao nosso modelo para extrair todas as informações de uma única vez!

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-query-function-review-wrapper.png" width="1200px">

#### A. Criando uma função para extrair todas as informações

In [0]:
CREATE OR REPLACE FUNCTION REVISAR_AVALIACAO(avaliacao STRING)
RETURNS STRUCT<
  produto_nome: STRING,
  produto_categoria: STRING,
  sentimento: STRING,
  resposta: STRING,
  resposta_motivo: STRING
>
RETURN FROM_JSON(
  REGEXP_REPLACE(
    AI_QUERY(
      'databricks-meta-llama-3-3-70b-instruct',
      CONCAT(
        'Analise a avaliação a seguir e retorne um JSON **válido e compacto**, sem texto adicional. ',
        'Campos obrigatórios: produto_nome, produto_categoria, sentimento, resposta, resposta_motivo. ',
        'Formato exato: {"produto_nome":"","produto_categoria":"","sentimento":"","resposta":"","resposta_motivo":""} ',
        'Avaliação: ', avaliacao
      )
    ),
    '\\n', ''  -- remove quebras de linha
  ),
  'STRUCT<
    produto_nome: STRING,
    produto_categoria: STRING,
    sentimento: STRING,
    resposta: STRING,
    resposta_motivo: STRING
  >'
);

#### B. Testando a análise das avaliações

In [0]:
SELECT 
  REVISAR_AVALIACAO(
    'Comprei um aspirador de pó da marca Electrolux e fiquei bem decepcionada. 
     O produto faz muito barulho e a sucção é fraca. 
     Esperava mais pela faixa de preço. 
     Gostaria que a empresa entrasse em contato para resolver o problema.'
  ) AS resultado;

#### C. Analisando todas as avaliações

In [0]:
CREATE OR REPLACE TABLE avaliacoes_revisadas AS
SELECT *, resultado.*
FROM (
  SELECT 
    avaliacao,
    REVISAR_AVALIACAO(avaliacao) AS resultado
  FROM avaliacoes
  LIMIT 10
);

SELECT * FROM avaliacoes_revisadas

Agora, todos os nossos usuários podem aproveitar nossa função que foi cuidadosamente preparada para analisar nossas avaliações de produtos.

E podemos escalar esse processo facilmente aplicando essa função sobre todo o nosso conjunto de dados!

## Exercício 02.03 - Gerando uma sugestão de resposta

Com todas as informações extraídas, podemos aproveitá-las para gerar sugestões de respostas personalizadas para acelerar o trabalho dos nossos times de atendimento.

Outro ponto interessante é que, nesse processo, podemos aproveitar outras **informações estruturadas** que já tenhamos no nosso ambiente, como dados demográficos, psicográficos e o histórico de compras, para customizar ainda mais nossas respostas!

Vamos ver como fazer isso!

### A. Criando uma função para gerar um exemplo de resposta

In [0]:
CREATE OR REPLACE FUNCTION GERAR_RESPOSTA(
    nome STRING,
    sobrenome STRING,
    num_pedidos INT,
    produto STRING,
    motivo STRING
)
RETURNS TABLE(resposta STRING)
COMMENT 'Caso o cliente demonstre insatisfação com algum produto, use esta função para gerar uma resposta personalizada'
RETURN 
SELECT AI_QUERY(
    'databricks-meta-llama-3-3-70b-instruct',
    CONCAT(
        'Você é um assistente virtual responsável por gerar uma resposta amigável e profissional para um cliente que está insatisfeito. ',
        'Seu objetivo é acalmar o cliente, reconhecer o problema e, se aplicável, sugerir uma solução ou troca dentro da política da empresa. ',
        'Evite gerar informações falsas ou inventar detalhes sobre o cliente ou pedidos. ',
        'Controle o tamanho da mensagem para que seja clara e objetiva (máx. 150 palavras). ',
        'Use um tom cordial, empático e profissional. ',
        'Dados do cliente: Nome: ', gerar_resposta.nome, ' ', gerar_resposta.sobrenome, 
        ', Número de pedidos anteriores: ', CAST(gerar_resposta.num_pedidos AS STRING), '. ',
        'Produto: ', gerar_resposta.produto, '. ',
        'Motivo da insatisfação: ', gerar_resposta.motivo, '. ',
        'Gere uma resposta personalizada considerando estas informações.'
    )
);

### B. Gerando respostas automatizadas para todas as avaliações negativas

In [0]:
CREATE OR REPLACE TABLE avaliacoes_revisadas AS
SELECT 
  a.id_avaliacao,
  a.id_cliente,
  c.nome,
  c.sobrenome,
  c.num_pedidos,
  a.avaliacao,
  REVISAR_AVALIACAO(a.avaliacao) AS resultado
FROM avaliacoes a
JOIN clientes c ON a.id_cliente = c.id_cliente;

SELECT * FROM avaliacoes_revisadas

In [0]:
CREATE OR REPLACE TABLE respostas AS
SELECT 
  r.id_avaliacao,
  r.id_cliente,
  r.nome,
  r.sobrenome,
  r.num_pedidos,
  r.resultado.produto_nome AS produto_nome,
  r.resultado.produto_categoria AS produto_categoria,
  r.resultado.sentimento AS sentimento,
  r.resultado.resposta_motivo AS resposta_motivo,
  CASE
    WHEN LOWER(r.resultado.sentimento) = 'negativo' THEN (
      SELECT * FROM GERAR_RESPOSTA(
        r.nome,
        r.sobrenome,
        r.num_pedidos,
        r.resultado.produto_nome,
        r.resultado.resposta_motivo
      )
    )
    ELSE NULL
  END AS rascunho
FROM avaliacoes_revisadas r;

SELECT * FROM respostas

# Parabéns!

Você concluiu o laboratório de **Extração de informações e geração de texto**!

Agora, você já sabe como utilizar a Foundation Models, Playground e AI Functions para analisar o sentimento e identificar entidades em avaliações de produtos de forma simples e escalável!